In [1]:
import os
os.system('pip install pandas')
os.system('pip install scikit-learn')
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

In [2]:
#df = pd.read_csv('data.csv')
#labels = pd.read_csv('labels.csv', header = None)

df = pd.read_csv('/data/p_dsi/big_data_scaling_sp23/project/Clustering/data.csv')
labels = pd.read_csv('/data/p_dsi/big_data_scaling_sp23/project/Clustering/labels.csv', header = None)

In [3]:
df.head()

,contigname,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31
0,S1CNODE_1_length_1066814_cov_33.542828,1.878896,-0.813271,1.625060,-3.371945,-1.619152,-1.975395,-2.487455,-2.923170,-0.548747,...,2.414434,-0.476510,2.493267,0.309539,-0.472919,0.780475,-2.704534,1.759772,0.593364,1.890912
1,S1CNODE_2_length_795377_cov_18.519996,-1.075560,-0.483019,1.201002,-3.024857,-1.248008,-0.490532,-1.301426,-1.785879,0.022432,...,1.582417,2.637151,-0.529287,1.257452,-0.711342,-0.848506,-1.638207,2.063375,1.731241,0.965337
2,S1CNODE_3_length_752009_cov_17.150503,-1.489421,2.832336,-0.372450,1.213937,-2.941387,-2.819255,-1.156072,1.554669,0.043990,...,1.223604,1.327222,0.149169,0.289314,-1.541078,-2.208405,-2.171267,3.151672,0.996597,0.103279
3,S1CNODE_4_length_594792_cov_17.061143,-1.183191,3.114740,-0.107469,0.861840,-2.730131,-2.948646,-1.036685,1.330220,0.186006,...,0.849787,1.654931,0.569403,-0.012465,-1.718321,-1.865685,-2.188801,3.503557,1.030231,0.269825
4,S1CNODE_5_length_504974_cov_23.783496,-0.066794,0.629984,0.480252,1.089288,-0.779640,-2.007908,-0.826030,2.128111,0.030095,...,0.454208,-1.236633,-0.981003,2.523260,-0.399886,-0.788499,-1.869508,2.635718,0.982824,0.948793


In [4]:
labels.head()

,0,1
0,S5CNODE_26938_length_4240_cov_11.524731,evo_1220_W.025
1,S5CNODE_257337_length_1143_cov_4.676471,evo_1220_W.003
2,S5CNODE_33322_length_3729_cov_10.363909,evo_1220_W.003
3,S1CNODE_25209_length_4364_cov_9.058946,evo_1220_Q.019
4,S1CNODE_25035_length_4387_cov_3.682595,evo_1048987.039


In [5]:
df_sampled = df
labels_sampled = labels

In [6]:
labels_sampled_2 = labels_sampled.copy()

In [7]:
le = LabelEncoder()
labels_sampled_2[1] = le.fit_transform(labels_sampled_2[1])

In [8]:
new_names = {0 : 'contigname'}
labels_sampled_2 = labels_sampled_2.rename(columns=new_names)

In [9]:
labels_sampled_2.head()

,contigname,1
0,S5CNODE_26938_length_4240_cov_11.524731,36
1,S5CNODE_257337_length_1143_cov_4.676471,35
2,S5CNODE_33322_length_3729_cov_10.363909,35
3,S1CNODE_25209_length_4364_cov_9.058946,34
4,S1CNODE_25035_length_4387_cov_3.682595,33


In [10]:
def z_score(df):
    df_std = df.copy()
    for column in df_std.columns:
        df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()    
    return df_std  
df_sampled_num = df_sampled.drop(['contigname'], axis = 1)
df_standardized = z_score(df_sampled_num)
df_standardized.head()

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31
0,1.608004,-0.852467,0.265687,-2.155313,-0.285824,-1.295209,-1.576420,-2.089835,-0.525996,-0.600381,...,0.917570,-0.249013,1.532652,-0.375063,0.141832,0.560343,-1.114338,0.014136,0.182572,1.559192
1,-0.745194,-0.613047,0.001085,-1.931316,-0.061808,-0.221289,-0.505409,-1.513415,0.079835,-0.947843,...,0.350481,1.967561,-0.483583,0.133279,-0.054644,-0.394379,-0.399180,0.213713,1.027652,0.845277
2,-1.074830,1.790460,-0.980712,0.804237,-1.083897,-1.905527,-0.374151,0.179694,0.102700,-0.161292,...,0.105920,1.035040,-0.031010,-0.385909,-0.738397,-1.191396,-0.756690,0.929117,0.482046,0.180356
3,-0.830921,1.995193,-0.815370,0.577008,-0.956388,-1.999108,-0.266342,0.065935,0.253332,-0.207984,...,-0.148866,1.268331,0.249313,-0.547746,-0.884456,-0.990532,-0.768449,1.160433,0.507025,0.308816
4,0.058279,0.193838,-0.448646,0.723794,0.220889,-1.318724,-0.076116,0.470335,0.087963,0.849097,...,-0.418486,-0.790134,-0.784906,0.812100,0.202015,-0.359210,-0.554308,0.589949,0.471817,0.832517


In [13]:
df_standardizednp = df_standardized.to_numpy()

In [14]:
df_standardizednp

array([[ 1.60800386e+00, -8.52467064e-01,  2.65687140e-01, ...,
         1.41360197e-02,  1.82572328e-01,  1.55919177e+00],
       [-7.45193595e-01, -6.13047339e-01,  1.08515709e-03, ...,
         2.13712577e-01,  1.02765178e+00,  8.45277308e-01],
       [-1.07483020e+00,  1.79046030e+00, -9.80711829e-01, ...,
         9.29117356e-01,  4.82045551e-01,  1.80355882e-01],
       ...,
       [-3.30551284e-01,  2.32564404e-01,  2.57920633e-01, ...,
        -4.18669735e-01,  1.59190932e+00,  1.22560900e+00],
       [-1.02885554e+00,  1.60990409e+00, -8.02387837e-01, ...,
         7.15434937e-01,  6.41019626e-01,  2.35742165e+00],
       [ 7.28269967e-02,  4.11217497e-01,  1.83970029e-01, ...,
         1.06436752e-01, -7.44821322e-01, -4.32684882e-01]])

In [15]:
model = KMeans(745, random_state = 2023)
res = model.fit(df_standardizednp)

In [16]:
len(res.labels_)

10000

In [17]:
clusters = pd.Series(res.labels_)

In [18]:
df_final = pd.DataFrame(df_standardizednp)
df_final['contigname'] = df_sampled['contigname']
df_final['predicted_label'] = clusters
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,contigname,predicted_label
0,1.608004,-0.852467,0.265687,-2.155313,-0.285824,-1.295209,-1.576420,-2.089835,-0.525996,-0.600381,...,1.532652,-0.375063,0.141832,0.560343,-1.114338,0.014136,0.182572,1.559192,S1CNODE_1_length_1066814_cov_33.542828,59
1,-0.745194,-0.613047,0.001085,-1.931316,-0.061808,-0.221289,-0.505409,-1.513415,0.079835,-0.947843,...,-0.483583,0.133279,-0.054644,-0.394379,-0.399180,0.213713,1.027652,0.845277,S1CNODE_2_length_795377_cov_18.519996,133
2,-1.074830,1.790460,-0.980712,0.804237,-1.083897,-1.905527,-0.374151,0.179694,0.102700,-0.161292,...,-0.031010,-0.385909,-0.738397,-1.191396,-0.756690,0.929117,0.482046,0.180356,S1CNODE_3_length_752009_cov_17.150503,717
3,-0.830921,1.995193,-0.815370,0.577008,-0.956388,-1.999108,-0.266342,0.065935,0.253332,-0.207984,...,0.249313,-0.547746,-0.884456,-0.990532,-0.768449,1.160433,0.507025,0.308816,S1CNODE_4_length_594792_cov_17.061143,717
4,0.058279,0.193838,-0.448646,0.723794,0.220889,-1.318724,-0.076116,0.470335,0.087963,0.849097,...,-0.784906,0.812100,0.202015,-0.359210,-0.554308,0.589949,0.471817,0.832517,S1CNODE_5_length_504974_cov_23.783496,634


In [24]:
df_joined = df_final.merge(labels_sampled_2, how = "inner", on = "contigname")

In [27]:
df_joined.head()

,0,1_x,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,contigname,predicted_label,1_y
0,-0.729194,-0.911694,0.770791,-0.878645,-1.470056,0.728544,-0.652589,-1.363606,-0.447704,0.178330,...,-1.004890,0.241155,1.395973,-0.465422,0.864917,0.400291,-0.508800,S1CNODE_2434_length_20967_cov_15.522475,338,31
1,0.245361,0.726924,-1.604655,0.176050,0.489850,1.694621,-0.197765,1.330974,-0.235256,-0.345955,...,-0.842782,-0.259134,1.587690,0.037234,1.858864,-0.289926,-0.794406,S1CNODE_2596_length_19997_cov_8.524571,626,3
2,-1.602717,-1.090540,1.691077,0.772205,1.378140,0.366170,-0.632962,0.084872,1.158306,-1.065722,...,-0.725195,0.469492,0.940587,1.094031,0.767810,-0.374129,-0.988059,S1CNODE_2607_length_19915_cov_21.171652,374,13
3,-0.098578,-2.054161,-0.416001,1.048189,1.513057,0.893738,0.180503,-0.034174,0.047663,1.154688,...,0.351977,0.447588,0.938134,1.346844,1.367438,-1.300103,-1.036785,S1CNODE_2683_length_19530_cov_6.421412,345,3
4,0.977577,-1.160592,-1.080577,0.573227,0.889666,1.225478,0.030036,0.325807,0.362332,0.635935,...,0.568452,-0.092169,1.506611,1.542231,1.299616,-1.492809,-1.020770,S1CNODE_2712_length_19349_cov_6.854307,522,3


In [28]:
print(adjusted_rand_score(df_joined['predicted_label'], df_joined['1_y']))

0.03615867276401149